In [46]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [47]:
# Load the CSV file into a DataFrame
df = pd.read_csv(r'C:\Users\nicholassng\OneDrive - Singapore Institute Of Technology\Desktop\datathon\data.csv')

In [48]:

columns_to_drop = ['sumins_grp_94baec', 'prempaid_inv_dcd836', 'prempaid_lh_d0adeb',
                   'prempaid_gi_a10d1b', 'prempaid_gi_29d435', 'prempaid_gi_856320',
                   'prempaid_gi_058815', 'prempaid_32c74c', 'ape_gi', 'sumins_gi',
                   'prempaid_d0adeb', 'prempaid_gi', 'sumins_d0adeb', 'sumins_e22a6a',
                   'ape_d0adeb', 'prempaid_gi_42e115', 'prempaid_ltc_1280bf', 'sumins_32c74c',
                   'sumins_gi_058815', 'ape_ltc_1280bf', 'ape_gi_42e115', 'ape_inv_dcd836',
                   'ape_lh_d0adeb', 'ape_gi_a10d1b', 'ape_gi_29d435', 'ape_gi_856320',
                   'ape_gi_058815', 'ape_32c74c', 'sumins_gi_42e115', 'sumins_ltc_1280bf',
                   'sumins_inv_dcd836', 'sumins_lh_d0adeb', 'sumins_grp_22decf',
                   'sumins_gi_a10d1b', 'sumins_gi_29d435', 'sumins_lh_e22a6a', 'sumins_grp_e04c3a',
                   'sumins_gi_856320']
# Drop the specified columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)


df.dropna(subset=['flg_substandard'], inplace=True)
df.dropna(subset=['hh_size_est'], inplace=True)
df.dropna(subset=['race_desc'], inplace=True)
df = df[(df['pop_20'] != '') & (df['pop_20'] != 0)]
df = df[~df['ctrycode_desc'].isin(["unknown country code", ""])] # drop blanks and unknown country code
df = df.filter(regex=r'^(?!f_hold)') #remove f_hold columns 

In [49]:
df['is_consent'] = (df['is_consent_to_mail'] + df['is_consent_to_email'] +
                    df['is_consent_to_call'] + df['is_consent_to_sms']) > 2

# Convert boolean values to 1s and 0s
df['is_consent'] = df['is_consent'].astype(int)

In [50]:
df['is_valid'] = (df['is_valid_dm'] + df['is_valid_email']) > 1

# Convert boolean values to 1s and 0s
df['is_valid'] = df['is_valid'].astype(int)

In [51]:
# replace '>4' as 5
df['hh_size_est'] = df['hh_size_est'].replace('>4', 5)

# Convert the column to integer type
df['hh_size_est'] = df['hh_size_est'].astype(int)

In [52]:
income_mapping = {
    'A.ABOVE200K': 1,
    'B.100K-200K': 2,
    'C.60K-100K': 3,
    'D.30K-60K': 4,
    'E.BELOW30K': 5
}

# Map the 'annual_income_est' column using the defined mapping
df['annual_income_est'] = df['annual_income_est'].map(income_mapping)

# Replace NaN values with 6
df['annual_income_est'].fillna(6, inplace=True)

# Convert the column to integer type
df['annual_income_est'] = df['annual_income_est'].astype(int)

C:\Users\nicholassng\AppData\Local\Temp\ipykernel_8376\3979558514.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['annual_income_est'].fillna(6, inplace=True)


In [53]:
ape_columns = df.filter(regex=r'^ape_(?!lapse_).*')
# Sum values along columns to create the new "total_ape" column
df['total_ape'] = ape_columns.sum(axis=1)

In [54]:
sumins_col = df.filter(like='sumins_')
df['total_sumins'] = sumins_col.sum(axis=1)

In [55]:
prempaid_col = df.filter(like='prempaid_')
df['total_prempaid'] = prempaid_col.sum(axis=1)

In [56]:
# Drop rows with 'None' values in the 'cltdob_fix' column
df = df[df['cltdob_fix'] != 'None']

# Split the "cltdob_fix" column into year, month, and day components
df['cltdob_fix'] = pd.to_datetime(df['cltdob_fix'], errors='coerce')  # Convert to datetime and handle invalid dates

# Calculate the current date
current_date = datetime.now()

# Define a function to calculate age
def calculate_age(row):
    if pd.isnull(row['cltdob_fix']):
        return pd.NA
    birth_date = row['cltdob_fix']
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return age

# Apply the function to calculate age and create the 'age' column
df['age'] = df.apply(calculate_age, axis=1)

# Drop the intermediate columns 'year', 'month', and 'day' if not needed
df.drop(['cltdob_fix'], axis=1, inplace=True)

In [57]:
label_encoder = LabelEncoder()
df['stat_flag_encoded'] = label_encoder.fit_transform(df['stat_flag'])

In [58]:
# Encode the 'cltsex_fix' column
df['cltsex_encoded'] = label_encoder.fit_transform(df['cltsex_fix'])

In [59]:
# Replace all blank values in 'f_ever_declined_la' with '0'
df['f_ever_declined_la'] = df['f_ever_declined_la'].fillna('0')

In [60]:
# df of cleaned data